### Contents

- [Configurations and Libraries](#Configurations-and-Libraries)
- [Functions](#Functions)

### Header

In [1]:
# import libraries

# maths
import scipy.stats as stats
import numpy as np
import pandas as pd
from haversine import haversine, Unit

# others
from tqdm import tqdm
import time
import datetime as dt
import re
import os

# web
import json

In [2]:
# file paths

folder = 'fitrec/'

raw_path = '../data/0_raw/' + folder
input_path = '../data/1_input/' + folder
clean_path = '../data/2_clean/' + folder
preprocess_path = '../data/3_preprocess/' + folder
output_path = '../data/4_output/' + folder

### Functions

In [3]:
# output smaller json file with incremental filenames e.g. xxx_123.json

def output_json(data,file_no):
    
    file_str = str(file_no)
    file_str = file_str.zfill(3)            
    out_path_2 = out_path + file_str + '.json'

    with open(out_path_2, 'w') as f:
        print(out_path_2)
        json.dump(data, f)

In [4]:
def compute_distance(record):
    
    #print(record['latitude'])
    #print(record['longitude'])
    
    for i in range(1,500,1):
        
        cur_loc = (record['latitude'][i],record['longitude'][i])
        prev_loc = (record['latitude'][i-1],record['longitude'][i-1])
        
        # metre
        dist = haversine(cur_loc,prev_loc) * 1000     
        record['distance'][i] = dist
        
        #print(i,prev_loc,cur_loc,dist)
        
    return record

In [13]:
def compute_speed(record):
    
    for i in range(1,500,1):
        
        # seconds
        time_diff = record['timestamp'][i] - record['timestamp'][i-1]
        
        if time_diff > 0:
            
            # metre/seconds
            speed = record['distance'][i] / time_diff
            
            # km/h
            record['speed'][i] = speed * (60*60/1000)
            
        elif time_diff == 0:
            record['speed'][i] = record['speed'][i-1]
            
        # time_diff < 0
        else:
            record['speed'][i] = record['speed'][i-1]
            print('error: time_diff < 0 for index',i)            
        
        #print(i,time_diff,speed)
        
    return record

### Import Data

In [6]:
# sample code from https://sites.google.com/eng.ucsd.edu/fitrec-project/home

# path = Path("data/")
# out_path = str(path / "processed_endomondoHR_proper.npy")
# data = np.load(out_path)[0]

In [7]:
# sample code from https://sites.google.com/eng.ucsd.edu/fitrec-project/home

# data = []
# #with gzip.open('endomondoHR.json.gz') as f:
# with open('endomondoHR_proper.json') as f:
#     for l in f:
#         data.append(eval(l))

In [8]:
# old method

#file = "processed_endomondoHR_proper.npy"
# file = "processed_endomondoHR_proper_interpolate.npy"

# path = input_path + folder + file

#processed_endomondoHR_proper = np.load(path,mmap_mode='r')[0]
#data = np.memmap(path,mode='r+')

In [9]:
# old method

# file = "endomondoHR_proper.json"
# path = input_path + file

# num_workout = 300000
# data = {}
# index = 1

# with open(path) as f:    

#     for l in tqdm(f):
        
#         if index > num_workout:
#             break     

#         data[index] = eval(l)        
#         index += 1

### Output Data

In [10]:
# old method

# file = "endomondoHR_proper_sample.json"
# path = clean_path + folder + file

# with open(path, 'w') as f:
#     json.dump(data, f)

### Import, Split and Output File

In [11]:
# select json file for splitting

#file_index = 0
#file_index = 1
file_index = 2
#file_index = 3
#file_index = 4

input_filepath = [
            "endomondoHR.json",
            "endomondoMeta.json",
            "endomondoHR_proper.json",
            "processed_endomondoHR_proper.npy",
            "processed_endomondoHR_proper_interpolate.npy"        
            ]


output_filepath = [
            "endomondoHR/endomondoHR_",
            "endomondoMeta/endomondoMeta_",
    
            #"endomondoHR_proper/endomondoHR_proper_",
            "endomondoHR_proper_dist_spd/endomondoHR_proper_dist_spd_",
    
            "processed_endomondoHR_proper_",
            "processed_endomondoHR_proper_interpolate_"        
            ]

# init filepaths

in_path = raw_path + input_filepath[file_index]
out_path = input_path + output_filepath[file_index]

print(in_path)
print(out_path)

../data/0_raw/fitrec/endomondoHR_proper.json
../data/1_input/fitrec/endomondoHR_proper_dist_spd/endomondoHR_proper_dist_spd_


In [14]:
# import large json file and split into smaller json files (1000 jsons each)

if '.json' in in_path:

    data = {}
    index = 1
    file_no = 1

    with open(in_path) as f:

        for l in f:

            data[index] = eval(l) 
                
            if 'speed' not in data[index].keys():

                data[index]['distance'] = [0] * 500
                data[index]['speed'] = [0] * 500
                
                data[index] = compute_distance(data[index])
                data[index] = compute_speed(data[index])

            # each file will contain 1000 jsons
            if index % 1000 == 0:            
                output_json(data,file_no)

                data = {}                
                file_no += 1

            index += 1

        # last file contains less than 1000 jsons
        if index % 1000 != 0:        
            output_json(data,file_no)            

../data/1_input/fitrec/endomondoHR_proper_dist_spd/endomondoHR_proper_dist_spd_001.json
error: time_diff < 0 for index 264
../data/1_input/fitrec/endomondoHR_proper_dist_spd/endomondoHR_proper_dist_spd_002.json
../data/1_input/fitrec/endomondoHR_proper_dist_spd/endomondoHR_proper_dist_spd_003.json
error: time_diff < 0 for index 155
../data/1_input/fitrec/endomondoHR_proper_dist_spd/endomondoHR_proper_dist_spd_004.json
error: time_diff < 0 for index 361
../data/1_input/fitrec/endomondoHR_proper_dist_spd/endomondoHR_proper_dist_spd_005.json
../data/1_input/fitrec/endomondoHR_proper_dist_spd/endomondoHR_proper_dist_spd_006.json
../data/1_input/fitrec/endomondoHR_proper_dist_spd/endomondoHR_proper_dist_spd_007.json
error: time_diff < 0 for index 281
../data/1_input/fitrec/endomondoHR_proper_dist_spd/endomondoHR_proper_dist_spd_008.json
../data/1_input/fitrec/endomondoHR_proper_dist_spd/endomondoHR_proper_dist_spd_009.json
error: time_diff < 0 for index 44
error: time_diff < 0 for index 100

../data/1_input/fitrec/endomondoHR_proper_dist_spd/endomondoHR_proper_dist_spd_078.json
error: time_diff < 0 for index 2
../data/1_input/fitrec/endomondoHR_proper_dist_spd/endomondoHR_proper_dist_spd_079.json
../data/1_input/fitrec/endomondoHR_proper_dist_spd/endomondoHR_proper_dist_spd_080.json
error: time_diff < 0 for index 45
../data/1_input/fitrec/endomondoHR_proper_dist_spd/endomondoHR_proper_dist_spd_081.json
../data/1_input/fitrec/endomondoHR_proper_dist_spd/endomondoHR_proper_dist_spd_082.json
../data/1_input/fitrec/endomondoHR_proper_dist_spd/endomondoHR_proper_dist_spd_083.json
../data/1_input/fitrec/endomondoHR_proper_dist_spd/endomondoHR_proper_dist_spd_084.json
../data/1_input/fitrec/endomondoHR_proper_dist_spd/endomondoHR_proper_dist_spd_085.json
../data/1_input/fitrec/endomondoHR_proper_dist_spd/endomondoHR_proper_dist_spd_086.json
error: time_diff < 0 for index 152
../data/1_input/fitrec/endomondoHR_proper_dist_spd/endomondoHR_proper_dist_spd_087.json
error: time_diff <

../data/1_input/fitrec/endomondoHR_proper_dist_spd/endomondoHR_proper_dist_spd_158.json
../data/1_input/fitrec/endomondoHR_proper_dist_spd/endomondoHR_proper_dist_spd_159.json
../data/1_input/fitrec/endomondoHR_proper_dist_spd/endomondoHR_proper_dist_spd_160.json
../data/1_input/fitrec/endomondoHR_proper_dist_spd/endomondoHR_proper_dist_spd_161.json
error: time_diff < 0 for index 250
../data/1_input/fitrec/endomondoHR_proper_dist_spd/endomondoHR_proper_dist_spd_162.json
error: time_diff < 0 for index 67
error: time_diff < 0 for index 123
../data/1_input/fitrec/endomondoHR_proper_dist_spd/endomondoHR_proper_dist_spd_163.json
error: time_diff < 0 for index 39
../data/1_input/fitrec/endomondoHR_proper_dist_spd/endomondoHR_proper_dist_spd_164.json
../data/1_input/fitrec/endomondoHR_proper_dist_spd/endomondoHR_proper_dist_spd_165.json
error: time_diff < 0 for index 1
error: time_diff < 0 for index 131
../data/1_input/fitrec/endomondoHR_proper_dist_spd/endomondoHR_proper_dist_spd_166.json
..

In [ ]:
if '.npy' in in_path:
    
    #data = np.load(in_path,mmap_mode='r')
    #data = np.memmap(in_path,mode='r+')
    
    #data = np.memmap(in_path,mode='r+')
    data = np.memmap(in_path,mode='r+',dtype='uint8')